In [3]:
import pandas as pd
import numpy as np
import requests
import os
import time

from datetime import datetime as dt
from datetime import timedelta

_from = dt.now()
to = _from + timedelta(hours=10, minutes=5)

In [2]:
token = "" # Your Tiingo API token.
headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Token ' + token
        }
requests.get("https://api.tiingo.com/api/test/",
                                    headers=headers)

<Response [200]>

In [ ]:
# Make sure not to send requests to frequently to the Tiingo server
class counter:
    def __init__(self, wait_h, wait_m, wait_s, period_max):
        self.wait_h = wait_h
        self.wait_m = wait_m
        self.wait_s = wait_s
        self.wait = timedelta(hours=self.wait_h, minutes=self.wait_m, seconds=self.wait_s)
        
        self.period_max = period_max
        
        self.start = dt.now()
        self.to = self.start + self.wait
        self.count = 0
        
    def add(self):
        self.last_add = dt.now()
        if self.last_add > self.start+self.wait:
            self.reset()
        
        self.count+=1
        
        if self.count == self.period_max:
            print('Reach maximum request in a period, wait until %s' %self.to)
            time.sleep((self.to - dt.now()).seconds)
            self.reset()
   
    def reset(self):
        self.start = dt.now()
        self.to = self.start +  self.wait
        self.count = 0
        print('counter reset')

In [ ]:
def get_price(tickers, dates, dates_end, _counter):
    error_log = []

    for d1, d2 in zip(dates, dates_end):
        d1 = str(d1).split(' ')[0]
        d2 = str(d2).split(' ')[0]
        print(d1, d2)

        query = {'tickers':tickers,
                 'startDate': d1, 
                 'endDate': d2,
                 'resampleFreq': '1hour'}

        
        r = requests.get("https://api.tiingo.com/tiingo/crypto/prices", 
                         params = query,
                         headers=headers)
        
        _counter.add()
        print('hourly request count:',_counter.count)

        if len(r.json())==0:
            error_log.append([(tickers, d1,d2)])
            print('Empty Json')

        else:
            df=pd.DataFrame(r.json()[0]['priceData'])
            df=df[['date', 'open', 'high', 'low', 'close', 'volume', 'tradesDone', 'volumeNotional']]
            df.to_csv(filename, mode="a", header=False)
        
    return error_log
#     time.sleep(0.5)          

In [ ]:
def clean_data(filename):
    file = pd.read_csv(filename, header=None)
    file.columns = ['id', 'date', 'open', 'high', 'low', 'close', 'volume', 'tradesDone', 'volumeNotional']

    file.drop_duplicates('date', inplace=True)

    file.date = pd.to_datetime(file.date)
#     file = file.loc[:,['date', 'open']]
#     file.columns = ['Date', '%s Price'%coin]
    file.to_csv(filename)

In [ ]:
# fetch price data of coins needed
coins = ['ada', 'bch', 'bnb', 'bsv', 'btc', 'dot', 'eos', 'eth', 'link',
       'ltc', 'xrp']

_counter = counter(1,0,3, 450)
error_log = []
error_tick = []

for r in range(len(coins)):
    coin = coins.iloc[r,:]
    unit = 'usd'
    tickers = coin.ticker+unit
    filename="tiingo_%s.csv"%coin

    print(filename)
    print(tickers)
    
    
    start1 = coin.start
    start2 = start1 + timedelta(days=3)
    
    end1 = coin.end
    end2 = end1 + timedelta(days=3)
    
    dates = pd.date_range(start=start1, end=end1, freq="3D")
    dates_end = pd.date_range(start=start2, end=end2, freq="3D")

    
    error = get_price(tickers, dates, dates_end, _counter)
    error_log.append(error)
    try:
        clean_data(filename)
    except:
        error_tick.append(coin)
